In [ ]:
%pwd

In [ ]:
import pandas as pd
df1  = pd.read_csv("results_summarised/all_results_sort_06_04_full.csv")
df1["file"] = "all_results_sort_06_04_full.csv"
df2  = pd.read_csv("results_summarised/all_new_results_sort_14_05_full.csv")
df2["file"] = "all_new_results_sort_14_05_full.csv"
df3  = pd.read_csv("results_summarised/sort_different_depth_17_05_2.csv")
df3["file"] = "sort_different_depth_17_05_2.csv"
df = pd.concat([df1, df2, df3])
# df = pd.concat([pd.read_csv("results_summarised/all_results_sort_06_04_full.csv"),
#                 pd.read_csv("results_summarised/all_new_results_sort_14_05_full.csv"),
#                 pd.read_csv("results_summarised/sort_different_depth_17_05_2.csv")])

In [ ]:
df

In [ ]:
import ast
import pandas as pd

# Assuming df is your DataFrame and 'test_loss_list' is the column you're working with.

for index, row in df.iterrows():
    try:
        # Try to evaluate the row as a literal.
        pd.eval(row['test_loss_list'])
    except Exception as e:
        # If an exception occurs, print the index and the value that failed
        print(f"Failed at index {index}: {row['test_loss_list']}")
        print(f"Error: {e}")
        # Optionally, break after the first failure
        #break
import ast
# replace "nan" with None
df['test_loss_list'] = df['test_loss_list'].str.replace('nan', 'None')
df['loss_list'] = df['loss_list'].str.replace('nan', 'None')
#df['test_loss_list'] = df['test_loss_list'].apply(lambda s: pd.eval(s))
#df['loss_list'] = df['loss_list'].apply(lambda s: pd.eval(s))

df['test_loss_list'] = df['test_loss_list'].apply(ast.literal_eval)
df['loss_list'] = df['loss_list'].apply(ast.literal_eval)

# Create a new DataFrame for Plotly Express
rows_list = []
for _, row in df.iterrows():
    params = row.to_dict()
    test_loss_list = params.pop('test_loss_list')
    loss_list = params.pop('loss_list')
    for epoch, loss in enumerate(test_loss_list):
        params['epoch'] = epoch
        params['test_loss'] = loss
        params['train_loss'] = loss_list[epoch]
        rows_list.append(params.copy())  # Use .copy() to avoid modifying the original params

# Create a new DataFrame
df_long = pd.DataFrame(rows_list)
df_long = df_long.fillna("missing")
# make embedding_function "gaussian" or "none"

def convert_embedding(name):
    if "GaussianEmbedder" in name:
        return "gaussian"
    elif "PositionalEncoding" in name:
        return "positional"
    elif "RandomFeature" in name:
        return "fourier"
    else:
        return "none"
df_long["embedding"] = df_long["embedding_function"].apply(convert_embedding)

# remove Unnamed: 0
df_long = df_long.drop(columns=["Unnamed: 0", "job_id"])
#df_long[(df_long["embedding"] == "fourier") & (df_long["epoch"] == 100) & (df_long["kernel"] == "gaussian") & (df_long["n_layer"] == 8) & (df_long["holes"] == "{}") & (df_long["length_scale"] == 1.)][["iter", "train_loss", "test_loss"]].sort_values("iter")
# average over iter
# fill na with "missing"
df_long = df_long.fillna("missing")
# convert all to string except test_loss
# removing rows where test_loss is missing
df_long = df_long[df_long["test_loss"] != "missing"]
# same for epoch
df_long = df_long[df_long["epoch"] != "missing"]
df_long = df_long.applymap(str)
df_long["test_loss"] = df_long["test_loss"].astype(float)
df_long["epoch"] = df_long["epoch"].astype(int)
df_long["train_loss"] = df_long["train_loss"].astype(float)
cols = list(df_long.columns)
# # remove iter and test_loss
cols.remove("iter")
cols.remove("test_loss")
cols.remove("train_loss")
# check that we have 5 rows for each combination of the other columns
#assert all(df_long.groupby(cols).size() > 5)
#show groups with less than 5
#df_long.groupby(cols).size().sort_values()
#df_long = df_long.groupby(cols)["test_loss"].mean().reset_index()
# #add a column for the number of rows and the mean test_loss
# df_long = df_long.groupby(cols).agg(
#     test_loss=('test_loss', 'median'),  # Calculate mean test_loss
#     train_loss=('train_loss', 'median'),  # Calculate mean test_loss
#     count=('test_loss', 'count'),           # Count the number of entries used for the mean
#     std=('test_loss', 'std')
# ).reset_index()


In [ ]:
#df_revert = df_long[df_long["revert_embedding"] == "True"]
df_revert = df_long[df_long["attention_only"] == "False"]
print(np.unique(df_revert["n_layer"], return_counts=True))
print(np.unique(df_revert["n_embd"], return_counts=True))
print(np.unique(df_revert["n_head"], return_counts=True))
print(np.unique(df_revert["parametrize_std"], return_counts=True))
print(np.unique(df_revert["n_epochs"], return_counts=True))
print(np.unique(df_revert["bandwidth"], return_counts=True))



In [ ]:
import plotly.express as px
import numpy as np
filter_dic = {
    #"revert_embedding": "True"
    "attention_only": "True",
    "n_layer": 8,
    "n_embd": 128,
    "parametrize_std": "False",
    #"read_in_bias": "False",
    #"bandwidth": "[0.1171875]",
    "embedding": "gaussian",

    
}
df_plot = df_long.copy()
for key, value in filter_dic.items():
    df_plot = df_plot[np.isin(df_plot[key], [value, "missing"])]

# remove embedding fourier
df_plot = df_plot[df_plot["embedding"] != "fourier"]
df_plot = df_plot[np.isin(df_plot["bandwidth"], ['[0.1171875]', '[0.5859375]'])]

In [ ]:
group_cols = list(df_plot.columns.difference(["test_loss", "train_loss", "iter"]))
df_plot = df_plot.groupby(group_cols).agg(
    test_loss=('test_loss', 'median'),  # Calculate mean test_loss
    train_loss=('train_loss', 'median'),  # Calculate mean test_loss
    count=('test_loss', 'count'),           # Count the number of entries used for the mean
    std=('test_loss', 'std')
).reset_index()

In [ ]:
# show remaining non-unique columns
for col in group_cols:
    if len(np.unique(df_plot[col])) > 1:
        # check if missing is in the unique values
        if "missing" in np.unique(df_plot[col]) and len(np.unique(df_plot[col])) == 2:
            continue
        elif len(np.unique(df_plot[col])) < 100:
            print(col, np.unique(df_plot[col], return_counts=True))
        else:
            print(col, len(np.unique(df_plot[col])))

In [ ]:
df_plot["bandwidth"] = df_plot["bandwidth"].apply(lambda x: x.replace("[", "").replace("]", "").replace(",", ""))
# rename bandwidth
df_plot["bandwidth"] = df_plot["bandwidth"].apply(lambda x: f"{float(x):.2f}")
df_plot["Length scale"] = df_plot["bandwidth"]
df_plot["Revert Embedding"] = df_plot["revert_embedding"]
df_plot["Epoch"] = df_plot["epoch"]

# make a random plot
fig = px.scatter(df_plot,
           color = "Revert Embedding",
           facet_col="Length scale",
           x="Epoch",
              y="test_loss",
              hover_data="file",
              log_y=True,
)
fig.write_image("plots/random.pdf")

fig = px.scatter(df_plot,
           color = "Revert Embedding",
           facet_col="Length scale",
           x="Epoch",
              y="test_loss",
              hover_data="file",
              log_y=True,
)

fig.update_layout(
      #title="Test loss over epochs",
      #xaxis_title="Epoch",
      yaxis_title="Test loss",
      # bigger font
      font=dict(
          size=18,
      ),
   )

# save as pdf
fig.write_image("plots/revert_embeddings.pdf")


# with mlp

In [ ]:
import plotly.express as px
import numpy as np
filter_dic = {
    #"revert_embedding": "True"
    "attention_only": "False",
    "n_layer": 4,
    "n_embd": 256,
    #"parametrize_std": "False",
    #"read_in_bias": "False",
    #"bandwidth": "[0.1171875]",
    #"embedding": "gaussian",

    
}
df_plot = df_long.copy()
for key, value in filter_dic.items():
    df_plot = df_plot[np.isin(df_plot[key], [value, "missing"])]

# remove embedding fourier
#df_plot = df_plot[df_plot["embedding"] != "fourier"]
#df_plot = df_plot[np.isin(df_plot["bandwidth"], ['[0.1171875]', '[0.5859375]'])]
group_cols = list(df_plot.columns.difference(["test_loss", "train_loss", "iter"]))
df_plot = df_plot.groupby(group_cols).agg(
    test_loss=('test_loss', 'median'),  # Calculate mean test_loss
    train_loss=('train_loss', 'median'),  # Calculate mean test_loss
    count=('test_loss', 'count'),           # Count the number of entries used for the mean
    std=('test_loss', 'std')
).reset_index()
# show remaining non-unique columns
for col in group_cols:
    if len(np.unique(df_plot[col])) > 1:
        # check if missing is in the unique values
        if "missing" in np.unique(df_plot[col]) and len(np.unique(df_plot[col])) == 2:
            continue
        elif len(np.unique(df_plot[col])) < 100:
            print(col, np.unique(df_plot[col], return_counts=True))
        else:
            print(col, len(np.unique(df_plot[col])))
#df_plot["bandwidth"] = df_plot["bandwidth"].apply(lambda x: x.replace("[", "").replace("]", "").replace(",", ""))
# rename bandwidth
#df_plot["bandwidth"] = df_plot["bandwidth"].apply(lambda x: f"{float(x):.2f}")
df_plot["Length scale"] = df_plot["bandwidth"]
df_plot["Revert Embedding"] = df_plot["revert_embedding"]
df_plot["Epoch"] = df_plot["epoch"]

# make a random plot
# fig = px.scatter(df_plot,
#            color = "Revert Embedding",
#            facet_col="Length scale",
#            x="Epoch",
#               y="test_loss",
#               hover_data="file",
#               log_y=True,
# )
#fig.write_image("plots/random.pdf")

fig = px.scatter(df_plot,
           color = "embedding",
           facet_col="Length scale",
           x="Epoch",
              y="test_loss",
              hover_data="file",
              log_y=True,
)

fig.update_layout(
      #title="Test loss over epochs",
      #xaxis_title="Epoch",
      yaxis_title="Test loss",
      # bigger font
      font=dict(
          size=18,
      ),
   )

# save as pdf
#fig.write_image("plots/revert_embeddings.pdf")
